<a href="https://colab.research.google.com/github/tiwaripari/ML/blob/main/Genre_SpotifyClassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np



In [45]:
df_2000 = pd.read_csv("/content/Spotify-2000.csv")
df_10 = pd.read_csv('/content/top10s.csv', encoding='latin1') # the engine needs to be changed otherwise UTF-8 error occurs
# df_top10s = pd.read_csv("/content/top10s.csv", engine = 'python')
df_2000.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59


In [46]:
df_10

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,599,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019,104,66,61,-7,20,16,176,1,3,75
599,600,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019,95,79,75,-6,7,61,206,21,12,75
600,601,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019,136,76,53,-5,9,65,260,7,34,70
601,602,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019,114,79,60,-6,42,24,217,1,7,69


In [48]:
 len(df_10["top genre"].unique())

50

In [49]:
# check data types using df.dtype
df_2000["Top Genre"].value_counts()

df_2000.drop(columns = ['Index', 'Title', 'Artist', 'Year'], inplace = True)


In [50]:
 df_10["top genre"].value_counts()

dance pop                    327
pop                           60
canadian pop                  34
barbadian pop                 15
boy band                      15
electropop                    13
british soul                  11
big room                      10
canadian contemporary r&b      9
neo mellow                     9
art pop                        8
complextro                     6
hip pop                        6
australian dance               6
atl hip hop                    5
australian pop                 5
edm                            5
permanent wave                 4
latin                          4
hip hop                        4
colombian pop                  3
tropical house                 3
brostep                        2
downtempo                      2
electronic trap                2
belgian edm                    2
escape room                    2
folk-pop                       2
electro                        2
indie pop                      2
detroit hi

In [51]:
df_10.drop(columns = ['Unnamed: 0', 'title', 'artist', 'year'], inplace = True)

In [52]:
df_10.columns = df_2000.columns # setting column names as each other
df = df_2000.append(df_10, ignore_index = True)

<ipython-input-52-074292984086>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_2000.append(df_10, ignore_index = True)


In [53]:
attributes = df.columns[1:]
for attribute in attributes:
    temp = df[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df[attribute][instance] = float(temp[instance].replace(',',''))


<ipython-input-53-350e4e455ed6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[attribute][instance] = float(temp[instance].replace(',',''))


In [54]:
# first extracting the genre columns
# getting rid of white spaces and turning it all into lower cases
genre = (df["Top Genre"].str.strip()).str.lower()

In [55]:
# function to split the genre column
def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ")
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')

In [56]:
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ")).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)


In [58]:
len(genre_m1.unique())

81

In [59]:
genre_m1.value_counts()

rock             835
pop              775
standards        123
metal             93
indie             80
                ... 
ambient            1
soundtrack         1
rock-and-roll      1
vocal              1
r&b                1
Name: Top Genre, Length: 81, dtype: int64

In [60]:
unique = genre_m1.unique()
to_remove = []

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 10: # 10 was arbitrarily chosen
        to_remove += [genre]
len(to_remove)

54

In [61]:
df['Top Genre'] = genre_m1
df

,Top Genre,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,standards,157,30,53,-14,11,68,201.0,94,3,71
1,rock,135,79,50,-11,17,81,207.0,17,7,39
2,hip,168,69,66,-9,7,52,341.0,2,17,69
3,metal,173,96,43,-4,3,37,269.0,0,4,76
4,rock,106,82,58,-5,10,87,256.0,1,3,59
...,...,...,...,...,...,...,...,...,...,...,...
2592,pop,104,66,61,-7,20,16,176,1,3,75
2593,pop,95,79,75,-6,7,61,206,21,12,75
2594,pop,136,76,53,-5,9,65,260,7,34,70
2595,pop,114,79,60,-6,42,24,217,1,7,69


In [62]:
df.set_index(["Top Genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df.drop(index = str(name), inplace = True)

df["Top Genre"].value_counts()

rock          835
pop           775
standards     123
metal          93
indie          80
soul           56
cabaret        51
wave           44
invasion       36
hip            33
europop        27
folk           26
mellow         26
dance          22
uk             22
band           21
room           19
country        18
funk           18
disco          18
electropop     16
americana      13
britpop        11
gold           11
beat           11
limburg        10
edm            10
Name: Top Genre, dtype: int64

In [63]:
df.dtypes

Top Genre                 object
Beats Per Minute (BPM)     int64
Energy                     int64
Danceability               int64
Loudness (dB)              int64
Liveness                   int64
Valence                    int64
Length (Duration)         object
Acousticness               int64
Speechiness                int64
Popularity                 int64
dtype: object

In [65]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)
# training set
X_train = train_set.values[:,1:]
y_train = train_set.values[:,0]

# test set
X_test = test_set.values[:,1:]
y_test = test_set.values[:,0]

from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(X_train)

# Standard Scaler
X_train_ST = standard_scaler.transform(X_train)
X_test_ST = standard_scaler.transform(X_test)

# obtaining all unique classes
unique = np.unique(y_train)

In [66]:
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
# 1 hot encoding
y_test_1hot = label_binarize(y_test, classes = unique)
y_train_1hot = label_binarize(y_train, classes = unique)

# labelling
y_test_label = LabelEncoder()


In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier


In [68]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

In [111]:
# training the models
model_method1 = LogisticRegression(multi_class = 'ovr').fit(X_train_ST, y_train)

# getting predictions
predictions_method1 = model_method1.predict(X_test_ST)

model = RandomForestClassifier(random_state = 42, min_samples_split = 5)
model.fit(X_train, y_train)
pred = model.predict(X_test_ST)

accuracy = accuracy_score(y_test, predictions_method1)
print(accuracy)
from sklearn.metrics import confusion_matrix
print(f1_score(y_test, pred, labels = unique, average = 'micro' ))


0.51340206185567
0.2927835051546392


In [112]:
kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

In [113]:
cv_score = cross_val_score(model, X_train_ST, y_train, cv = kfold, scoring = 'accuracy')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


In [114]:
print('%s: %f (%f)' % (name,cv_score.mean(), cv_score.std()))

r&b: 0.528866 (0.027778)


In [115]:
from feature_genre import *

In [116]:
token = get_token()


In [117]:
track_id = get_track_id("Abhi Mujhe Mein Kahin", token)

In [118]:
features_1 = get_features(track_id, token)

In [119]:
pred_1 = model.predict(features_1)
pred_1[0]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


'rock'